### Vergleich zwischen unverarbeiteten und verarbeiteten DICOM-Daten

In [ ]:
import os
import cv2
import pydicom
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# Angepasster absoluter DICOM-Pfad zur Patientin AMBL-008

dcm_image_folder = "/mnt/e/Advanced-MRI-Breast-Lesions/DICOM Images/manifest-1713182663002/Advanced-MRI-Breast-Lesions/AMBL-008/03-06-2004-NA-MRI BREASTS - Delayed contrast-48705/" \
"600.000000-Registered Ax T2 FSE-88003"
dcm_roi_folder = "/mnt/e/Advanced-MRI-Breast-Lesions/DICOM Images/manifest-1713182663002/Advanced-MRI-Breast-Lesions/AMBL-008/03-06-2004-NA-MRI BREASTS - Delayed contrast-48705/" \
"500.000000-ROI-25364"

save_path = "images/maske.png"
Z_TOLERANCE = 0.5

# ---------------------------------------------------------
# Lade ROI-Masken und Z-Koordinaten

def load_roi_mask():
    for file in os.listdir(dcm_roi_folder):
        if file.lower().endswith(".dcm"):
            roi_path = os.path.join(dcm_roi_folder, file)
            ds = pydicom.dcmread(roi_path)
            seg = ds.pixel_array
            zs = [float(fg.PlanePositionSequence[0].ImagePositionPatient[2])
                  for fg in ds.PerFrameFunctionalGroupsSequence]
            order = np.argsort(zs)
            return seg[order], [zs[i] for i in order]
    return None, None

# ---------------------------------------------------------
# Vorverarbeitung 

def preprocess_dicom(path):
    ds = pydicom.dcmread(path)
    arr = ds.pixel_array.astype(np.float32)
    z = float(ds.ImagePositionPatient[2])
    original = arr.copy()

    arr = cv2.GaussianBlur(arr, (5, 5), sigmaX=1)
    min_val, max_val = np.percentile(arr, 1), np.percentile(arr, 99)
    arr = np.clip(arr, min_val, max_val)
    arr = (arr - min_val) / (max_val - min_val + 1e-6)
    processed = np.rot90(arr, 2)
    return original, processed, z

def match_mask(z_target, seg_arr, seg_zs):
    combined = np.zeros_like(seg_arr[0], dtype=np.uint8)
    for seg_z, seg in zip(seg_zs, seg_arr):
        if abs(seg_z - z_target) <= Z_TOLERANCE:
            combined |= (seg > 0).astype(np.uint8)
    return np.rot90(combined, 2)

# ---------------------------------------------------------
# Visualisierung

def generate_comparison_figure():
    seg_arr, seg_zs = load_roi_mask()
    if seg_arr is None:
        raise FileNotFoundError("Keine ROI-Daten gefunden")

    dcm_files = [os.path.join(dcm_image_folder, f) for f in os.listdir(dcm_image_folder) if f.lower().endswith(".dcm")]
    if not dcm_files:
        raise FileNotFoundError("Keine DICOM-Dateien gefunden")

    images, masks, originals = [], [], []

    for path in dcm_files:
        original, processed, z = preprocess_dicom(path)
        mask = match_mask(z, seg_arr, seg_zs)
        if np.mean(mask) > 0.001:
            images.append(processed)
            masks.append(mask)
            originals.append(original)

    if not images:
        raise ValueError("Keine validen Tumor-Slices gefunden")

    idx = np.argmax([np.sum(m) for m in masks])
    processed = images[idx]
    mask = masks[idx]
    original_unprocessed = originals[idx]

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original_unprocessed, cmap="gray")
    axes[0].axis("off")

    axes[1].imshow(processed, cmap="gray")
    axes[1].imshow(np.ma.masked_where(mask == 0, mask), cmap="spring", alpha=0.4)
    axes[1].axis("off")

    plt.tight_layout()
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=300)
    print(f"Vergleichsgrafik gespeichert unter: {save_path}")
    plt.show()

# ---------------------------------------------------------
# Ausführen

if __name__ == "__main__":
    generate_comparison_figure()
